In [3]:

import pandas as pd
import numpy as np
import sqlalchemy

from datetime import datetime
import time
import ta

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output


import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

defaut_path = "C:/Users/cc/Desktop/OpenclassRoom/Interface/DataBase/"

def database(Id , interval , path = defaut_path):
    engine = sqlalchemy.create_engine('sqlite:///'+path+'basefiat_{}.db'.format(interval))
    data = pd.read_sql(Id+'USDT' ,engine)
    data.set_index('time' , inplace=True)
    data['volume'] = pd.to_numeric(data['volume'])
    data = data[['open', 'high', 'low' , 'close' , 'volume', 'symbol', 'close_time']]
    return data


In [4]:
app = dash.Dash(__name__, title = "cedTradAnalytics")

## figure

In [5]:
fig_0 = subplots(nb_rows = 1 , nb_cols = 2)

fig_0.add_trace(
    go.Scatter(
        x = trade.index,
        y = trade["cum_ret"]
    )
)

NameError: name 'subplots' is not defined

In [ ]:
fig = subplots(nb_rows = 1 , nb_cols = 3)

# Recovery
fig.add_trace(
    go.Scatter(
        x = Benchmark.index,
        y = Benchmark["price_return"],
        name = "recovery"
    ),
    col = 1, row = 1
)

# Price
fig.add_trace(
    go.Scatter(
        x = Benchmark.index,
        y = Benchmark["price"],
        name = "price"
    ),
    col = 2, row = 1
)

plot_LSB(fig, col = 3 , row = 1 ,Benchmark = R.Trades, Trades = R.Trades_reduit,
             Trades_long = R.Trades_reduit_long, Trades_short = R.Trades_reduit_short)

Fig = go.FigureWidget(fig)

## layout_0

In [ ]:
layout_0 = html.Div([
    html.Div([
        
        html.Div([
            dcc.Graph(id = 'current_trade', figure = fig_0)
        ]),
        html.Div([
            dcc.Graph(id = "cppi", figure = fig_0)
        ])
    ],
             style = {'display' : 'inline-block', 'width' : '58%'}),
    
    html.Div([
        html.H3("Current trade info"),
        html.Br(),
        html.H4("Entry"),
        html.Div(id = "price"),
        html.Div(id = "amount"),
        html.Div(id = "units"),
        html.Br(),
        html.H4("Current"),
        html.Div(id = "price_c"),
        html.Div(id = "amount_c"),
        html.Div(id = "cum_ret")
    ],
             style = {'display' : 'inline-block', 'width' : '38%'})
])

## layout_1

In [ ]:
layout_1 = html.Div([
    html.Br(),
    dcc.Graph( figure = Fig)
])


In [ ]:
layout_2 = html.Div([
    # Stats_ return
    html.Div([
        html.Div(id = "cum_ret"),
        html.Div(id = "drawdown"),
        html.Div(id = "min_ret"),
        html.Div(id = "max_ret"),
        html.Div(id = "mean"),
        html.Div(id = "VaR"),
        html.Div(id = "CVaR")
    ]),
    # stats 
    html.Div([
        html.Div(id = "amount"),
        html.Div(id = "amount_win"),
        html.Div(id = "amount_loss"),
        html.Div(id = "win_rate"),
        html.Div(id = "sharpe_ratio"),
        html.Div(id = "Expectancy"),
    ]),
])

In [ ]:
app.layout = html.Div([
    layout_0,
    layout_1
])

In [ ]:
if __name__ == '__main__':
    app.run_server(debug = False)

## Test

In [ ]:

import pandas as pd
import numpy as np
import sqlalchemy
import datetime

import dash
import dash_core_components as dcc
import dash_html_components as html

import plotly.express as px
import plotly.graph_objects as go

from dash.dependencies import Input, Output


def Data(symbol, table):
    engine = sqlalchemy.create_engine(f'sqlite:///base_{symbol}.db')
    data = pd.read_sql(table, engine)
    data['date'] = pd.to_datetime(data['date'])
    data.set_index("date", inplace=True)
    data["cum_ret"] = data["cum_ret"].astype("float64")
    return data

symbol = "BTC"


app = dash.Dash(__name__)

app.layout = html.Div([
    
    html.Div([
        dcc.Slider(min = 0.5, max = 5,
                   step = 0.5, value = 1,
                   id = "interval-refresh")
    ],
             style = {"width" : "20%"}),
    
    html.Div(id='latest-timestamp', style={"padding": "20px"}),
    dcc.Interval(
            id='interval-component',
            interval=1 * 1000,
            n_intervals=0
    ),
    
    html.Div([
        dcc.Graph(id = "main")
    ])
])


@app.callback(
    [Output(component_id = "interval-component", component_property = "interval")],
    [Input("interval-refresh", "value")]
)
def update_refresh_rate(value):
    return [value * 1000]


@app.callback(
    [Output(component_id='latest-timestamp', component_property='children')],
    [Input('interval-component', 'n_intervals')]
)
def update_timestamp(interval):
    return [html.Span(f"Last updated: {datetime.datetime.now()}")]

@app.callback(
    Output(component_id = "main", component_property = "figure"),
    Input("interval-component", "n_intervals")
)
def update_graph(n):
    Trades = Data(symbol = symbol, table = "Trades")
    Trades_reduit = Data(symbol = symbol, table = "Trades_reduit")
    Long = Data(symbol = symbol, table = "Trades_long")
    Short = Data(symbol = symbol, table = "Trades_short")
    
    
    trades_line = go.Scatter(
        x = Trades.index ,
        y = Trades["cum_ret"],
        line = dict(color = 'blue' , width = 1.8),
        name = 'strategie'
        )
    
    price_line = go.Scatter(
        x = Trades.index ,
        y = Trades["price_cum_ret"],
        line = dict(color = 'black' , width = 1.8),
        name = 'price'
        )
    
    long_line = go.Scatter(
        x = Long.index ,
        y = Long["cum_ret"], 
        line = dict(color = 'green' , width = 1.8),
        name = 'long_line'
        )
    
    short_line = go.Scatter(
        x = Short.index ,
        y = Short["cum_ret"], 
        line = dict(color = 'red' , width = 1.8),
        name = 'short_line'
        )
    
    layout = go.Layout(
        height = 600 , width = 1500,
        template="simple_white"
        )
    
    fig = go.Figure(
        data = [trades_line, price_line, long_line, short_line],
        layout = layout
    )    
    return fig


In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

# Graphe

In [ ]:
import plotly.express as px
df = px.data.medals_long()

fig = px.bar(df, x="medal", y="count", color="nation", text_auto=True)
fig.show()

In [ ]:
import plotly.express as px
df = px.data.medals_long()

fig = px.bar(df, x="medal", y="count", color="nation",
             pattern_shape="nation", pattern_shape_sequence=[".", "x", "+"])
fig.show()

In [ ]:
import plotly.graph_objects as go
animals=['giraffes', 'orangutans', 'monkeys']

fig = go.Figure(data=[
    go.Bar(name='SF Zoo', x=animals, y=[20, 14, 23]),
    go.Bar(name='LA Zoo', x=animals, y=[12, 18, 29])
])
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [ ]:
import plotly.graph_objects as go
x = [1, 2, 3, 4]

fig = go.Figure()
fig.add_trace(go.Bar(x=x, y=[1, 4, 9, 16]))
fig.add_trace(go.Bar(x=x, y=[6, -8, -4.5, 8]))
fig.add_trace(go.Bar(x=x, y=[-15, -3, 4.5, -8]))
fig.add_trace(go.Bar(x=x, y=[-1, 3, -3, -4]))

fig.update_layout(barmode='relative', title_text='Relative Barmode')
fig.show()

In [6]:
import plotly.graph_objects as go

animals = ['giraffes']

trace_1 = go.Bar(
    name = 'SF Zoo', x = animals , y = [20, 14, 23], text = animals
)

trace_2 = go.Bar(
    name = 'LA Zoo', x = animals , y = [12, 18, 29], text = animals
)

fig = go.Figure(
    data = [trace_1, trace_2]
)
fig.update_layout(barmode = 'stack')

#fig.update_xaxes(range= [0, 3])
#fig.update_yaxes(range = [0, 100])

fig

In [7]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x = [1.5, 4.5],
        y = [0.75, 0.75],
        text = ["Unfilled Rectangle", "Filled Rectangle"],
        mode = "text"
    )
)

# set axes properties
fig.update_xaxes(range = [0, 7], showgrid = False)
fig.update_yaxes(range = [0, 3.5])

# Add shapes
fig.add_shape(type = "rect", 
              x0 = 1, y0 = 1, x1 = 2, y1 = 3,
              line = dict(color = "RoyalBlue"))

fig.add_shape(type = "rect", 
              x0 = 3, y0 = 1, x1 = 6, y1 = 2,
              line = dict(color = "RoyalBlue", width = 2),
              fillcolor = "LightSkyBlue")

fig.update_shapes(dict(xref = 'x', yref = 'y'))

fig

In [8]:
import plotly.graph_objects as go

fig = go.Figure()

# Create scatter trace of text labels
fig.add_trace(go.Scatter(
    x=[1.5, 3],
    y=[2.5, 2.5],
    text=["Rectangle reference to the plot",
          "Rectangle reference to the axes"],
    mode="text",
))

# Set axes properties
fig.update_xaxes(range=[0, 4])
fig.update_yaxes(range=[0, 4])

# Add shape
fig.add_shape(type = "rect",
              xref = "x", yref = "y",
              x0 = 2.5, y0 = 0,
              x1 = 3.5, y1 = 2,
              line = dict(
                  color = "RoyalBlue",
                  width = 3
              ),
              fillcolor="LightSkyBlue")



In [9]:
Fig = go.FigureWidget(fig)
Fig.update_layout(height = 800 , width =1500,
                  margin=dict(
                        l=5, r=0, b=10, t=30,
                        pad=1
                    ),
                  showlegend=True,
                  )
Fig.update_layout(legend=dict(
                        orientation="h",
                        yanchor="bottom",
                        y=1.02,
                        xanchor="right",
                        x=0.7
                    )
                  )

FigureWidget({
    'data': [{'mode': 'text',
              'text': [Rectangle reference to the plot, Rectangle…

In [10]:
import plotly.graph_objects as go

fig = go.Figure()

# Add scatter trace for line
fig.add_trace(go.Scatter(
    x=["2015-02-01", "2015-02-02", "2015-02-03", "2015-02-04", "2015-02-05",
       "2015-02-06", "2015-02-07", "2015-02-08", "2015-02-09", "2015-02-10",
       "2015-02-11", "2015-02-12", "2015-02-13", "2015-02-14", "2015-02-15",
       "2015-02-16", "2015-02-17", "2015-02-18", "2015-02-19", "2015-02-20",
       "2015-02-21", "2015-02-22", "2015-02-23", "2015-02-24", "2015-02-25",
       "2015-02-26", "2015-02-27", "2015-02-28"],
    y=[-14, -17, -8, -4, -7, -10, -12, -14, -12, -7, -11, -7, -18, -14, -14,
       -16, -13, -7, -8, -14, -8, -3, -9, -9, -4, -13, -9, -6],
    mode="lines",
    name="temperature"
))

# Add shape regions
fig.add_vrect(
    x0="2015-02-04", x1="2015-02-06",
    fillcolor="LightSalmon", opacity=0.5,
    layer="below", line_width=0,
),

fig.add_vrect(
    x0="2015-02-20", x1="2015-02-22",
    fillcolor="LightSalmon", opacity=0.5,
    layer="below", line_width=0,
)

fig.show()

In [11]:
import plotly.graph_objects as go

labels = ['Oxygen','Hydrogen','Carbon_Dioxide','Nitrogen']
values = [4500, 2500, 1053, 500]

Pie_chart = go.Pie(
    labels = labels, values = values
)

fig = go.Figure(
    data = [Pie_chart]
)

fig

In [12]:
import plotly.graph_objects as go

labels = ['Oxygen','Hydrogen','Carbon_Dioxide','Nitrogen']
values = [4500, 2500, 1053, 500]

# pull is given as a fraction of the pie radius
fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0, 0.2, 0])])
fig.show()

In [13]:
import plotly.graph_objects as go

fig =go.Figure(go.Sunburst(
    labels=["Eve", "Cain", "Seth", "Enos", "Noam", "Abel", "Awan", "Enoch", "Azura"],
    parents=["", "Eve", "Eve", "Seth", "Seth", "Eve", "Eve", "Awan", "Eve" ],
    values=[10, 14, 12, 10, 2, 6, 6, 4, 4],
))
fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))

fig.show()

In [14]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame([
    dict(Task="Job A", Start='2009-01-01', Finish='2009-02-28', Resource="Alex"),
    dict(Task="Job B", Start='2009-03-05', Finish='2009-04-15', Resource="Alex"),
    dict(Task="Job C", Start='2009-02-20', Finish='2009-05-30', Resource="Max")
])

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Resource", color="Resource")
fig.show()

In [ ]:
import plotly.express as px

df = px.data.gapminder().query("year == 2007")
fig = px.sunburst(df, path=['continent', 'country'], values='pop',
                  color='lifeExp', hover_data=['iso_alpha'])
fig.show()

In [ ]:
import plotly.express as px
data = dict(
    number=[39, 27.4, 20.6, 11, 2],
    stage=["Website visit", "Downloads", "Potential customers", "Requested price", "Invoice sent"])
fig = px.funnel(data, x='number', y='stage')
fig.show()

In [ ]:
import plotly.graph_objects as go
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']

fig = go.Figure(data=[go.Pie(labels=['Oxygen','Hydrogen','Carbon_Dioxide','Nitrogen'],
                             values=[4500,2500,1053,500])])
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
fig.show()


In [ ]:
import plotly.graph_objects as go
import plotly.express as px

data = [[1, 25, 30, 50, 1],
        [20, 1, 60, 80, 30],
        [30, 60, 1, 5, 20]]

fig = px.imshow(data,
                labels=dict(x="Day of Week", y="Time of Day", color="Productivity"),
                x=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],
                y=['Morning', 'Afternoon', 'Evening'])
fig.show()

In [ ]:
import plotly.graph_objects as go
import plotly.express as px

data = [[1, 25, 30, 50, 1]]

fig = px.imshow(data,
                labels=dict(x="Day of Week", y="Time of Day", color="Productivity"),
                x=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],
                y=['Morning'])pd
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    data = go.Heatmap(
        z = [[1, None, 30, 50, 1], [20, 1, 60, 80, 30], [30, 60, 1, -10, 20]],
        x=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'],
        y=['Morning', 'Afternoon', 'Evening'],
    )
)
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    go.Indicator(
        #mode = "number+gauge+delta",
        mode = "number+gauge+delta",
        gauge = {'shape' : 'bullet'},
        value = 220,
        delta = {'reference' : 300},
        domain = {'x' : [0, 1], 'y' : [0, 1]},
        title = {'text': 'Profit'}
    )
)
fig.update_layout(height = 250)
fig.show()

In [ ]:
import plotly.graph_objects as go


fig = go.Figure(
    go.Indicator(
        mode= "number+gauge+delta", value = 220,
        title = {'text' : "<b>Profit</b>"},
        delta = {'reference' : 200},
        gauge = {
            'shape' : 'bullet',
            'axis' : {'range' : [None, 300]},
            'threshold' : {
                'line' : {'color' : 'red', 'width' : 2},
                'thickness' : 0.75,
                'value' : 280
            },
            'steps' : [
                {'range' : [0, 150], 'color' : 'lightgray'},
                {'range' : [150, 250], 'color' : 'gray'}
            ],
            'bar' : {'color' : 'blue'}
        }
    )
)
fig.update_layout(height = 250)
fig.show()

In [ ]:
import plotly.graph_objects as go

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Waterfall(
        name = "20", orientation = "v",
        measure = ["relative", "relative", "total", "relative", "relative", "total"],
        x = ["Sales", "Consulting", "Net revenue", "Purchses", "Other expenses", "Profit before tax"],
        textposition = "outside",
        text = ["+60", "+80", "-40", "-20", "Total"],
        y = [60, 80, 0, -40, -20, 0],
        connector = {"line":{"color":"rgb(63, 63, 63)"}}
    )
)

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Waterfall(
    x = [["2016", "2017", "2017", "2017", "2017", "2018", "2018", "2018", "2018"],
        ["initial", "q1", "q2", "q3", "total", "q1", "q2", "q3", "total"]],
    measure = ["absolute", "relative", "relative", "relative", "total", "relative", "relative", "relative", "total"],
    y = [1, 2, 3, -1, None, 1, 2, -4, None],
    base = 1000
))

fig.show()


In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Waterfall(
        x = [["2016", "2017", "2017", "2017", "2017", "2018", "2018", "2018", "2018"], 
             ["initial", "q1", "q2", "q3", "total", "q1", "q2", "q3", "total"]],
        measure = ["absolute", "relative", "relative", "relative", "total", "relative", "relative", "relative", "total"],
        y = [10, 20, 30, -10, None, 10, 20, -40, None], base = 300,
        decreasing = {"marker":{"color":"Maroon", "line":{"color":"red", "width":2}}},
        increasing = {"marker":{"color":"Teal"}},
        totals = {"marker":{"color":"deep sky blue", "line":{"color":"blue", "width":3}}}
    )
)


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(
    go.Indicator(
        value = 200,
        delta = {'reference': 160},
        gauge = {
            'axis': {'visible': False}},
        domain = {'row': 0, 'column': 0})
              )

fig.add_trace(
    go.Indicator(
        value = 120,
        gauge = {
            'shape': "bullet",
            'axis' : {'visible': False}},
        domain = {'x': [0.05, 0.5], 'y': [0.15, 0.35]})
              )

fig.add_trace(
    go.Indicator(
        mode = "number+delta",
        value = 300,
        domain = {'row': 0, 'column': 1})
                )

fig.add_trace(
    go.Indicator(
        mode = "delta",
        value = 40,
        domain = {'row': 1, 'column': 1})
                )

fig.update_layout(
    grid = {'rows': 2, 'columns': 2, 'pattern': "independent"},
    template = {'data' : {'indicator': [{
        'title': {'text': "Speed"},
        'mode' : "number+delta+gauge",
        'delta' : {'reference': 90}}]
                         }}
    )


In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Indicator(
    mode = "number+delta",
    value = 400,
    number = {'prefix': "$"},
    delta = {'position': "top", 'reference': 320},
    domain = {'x': [0, 1], 'y': [0, 1]}))

fig.update_layout(paper_bgcolor = "lightgray")

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = 200,
    domain = {'x': [0, 0.5], 'y': [0, 0.5]},
    delta = {'reference': 400, 'relative': True, 'position' : "top"}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = 350,
    delta = {'reference': 400, 'relative': True},
    domain = {'x': [0, 0.5], 'y': [0.5, 1]}))

fig.add_trace(go.Indicator(
    mode = "number+delta",
    value = 450,
    title = {"text": "Accounts<br><span style='font-size:0.8em;color:gray'>Subtitle</span><br><span style='font-size:0.8em;color:gray'>Subsubtitle</span>"},
    delta = {'reference': 400, 'relative': True},
    domain = {'x': [0.6, 1], 'y': [0, 1]}))

fig.show()